# Run alternative versions of the FastScape landscape evolution model

Let's import some packages first (you can install them using ``conda``).

In [ ]:
import numpy as np
import xarray as xr
import xsimlab as xs

In [ ]:
xs.__version__

## Generate an alternative model from an existing model

Most of the time we don't want to create new models from scratch, but instead just change one or several components. The ``xarray-simlab`` framework allows to do that very easily.

Let's first import the basic (standard) model.

In [ ]:
from fastscape.models import basic_model

basic_model

The ``fastscape`` package also contains a bunch of ``xarray-simlab`` components (called processes) that you can reuse to create new models. In the example below, we create a model variant in which we use multiple direction flow routing:  

In [ ]:
from fastscape.processes import MultipleFlowRouter

mfd_model = basic_model.update_processes({'flow': MultipleFlowRouter})

Compared to ``basic_model``, you can see that ``mfd_model`` as one additional input:

In [ ]:
mfd_model

In [ ]:
mfd_model.visualize(show_inputs=True)

It is also possible to drop components. For example, disable diffusion:

In [ ]:
no_diff_model = basic_model.drop_processes('diffusion')

no_diff_model

``xarray-simlab`` doesn't allow creating models that break the workflow (graph of components). For example:

In [ ]:
basic_model.drop_processes('flow')

## Reuse model setups

Likewise, most of the time we want to slightly modify an existing setup instead of creating a new setup from srcatch. In this case too, ``xarray-simlab`` allows to easily reuse and tweak model setup, even when using different model variants.

Let's import below the setup that we have created in the ``run_basic_model`` notebook:

In [ ]:
in_ds = xr.load_dataset('basic_input.nc')

in_ds

As the ``xarray-simlab`` API takes and returns ``xarray.Datasets`` objects, a common and tidy worflow is chain operations, like updating the setup and run the model (and do some post-processing). Let's setup and run a simulation using multiple flow routing:

In [ ]:
# I need to fix this (output variable info should load from netcdf file)

out_vars = {
    'out': ['topography__elevation',
            'flow__drainage_area',
            'flow__basin'],
    None: ['boundary__border',
           'grid__x',
           'grid__y',
           'spl__chi']
}

In [ ]:
in_vars = {'flow__slope_exp': 1.5}

with mfd_model:
    out_ds = (
        in_ds.xsimlab.filter_vars()
             .xsimlab.update_vars(input_vars=in_vars, output_vars=out_vars)
             .xsimlab.run()
             .set_index(x='grid__x', y='grid__y')
    )
    
out_ds

Let's look at the topography:

In [ ]:
import hvplot.xarray
import matplotlib.pyplot as plt

out_ds.topography__elevation.hvplot.image(x='x', y='y',
                                          cmap=plt.cm.viridis,
                                          groupby='out')